In [36]:
import pandas as pd

# === Load raw data ===
df = pd.read_csv("Food_Inspections_20250216.csv")

# === Drop unstructured or irrelevant columns (optional but recommended) ===
df = df.drop(columns=["Violations", "Latitude", "Longitude", "Location"], errors="ignore")

# === Drop rows with missing values in essential fields ===
df = df.dropna()

# === Convert float columns to string for categorical matching (FDs/INDs prefer discrete values) ===
float_cols = df.select_dtypes(include=["float64"]).columns
df[float_cols] = df[float_cols].astype(int).astype(str)

# === Normalize all object columns (trim and lowercase for IND checking) ===
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].str.strip().str.lower()

# === Export cleaned version for FD/IND analysis ===
df.to_csv("Food_Inspections_Cleaned.csv", index=False)


In [37]:
import pandas as pd
import time
from tane import TANE, read_db

# === Step 1: Load cleaned dataset and prepare sample ===
df = pd.read_csv("Food_Inspections_Cleaned.csv")
columns_to_use = [
    "Inspection ID", "License #", "DBA Name", "Facility Type",
    "Risk", "City", "State", "Results"
]
df_sample = df[columns_to_use].head(2000)
sample_csv = "Food_Inspections_Sample.csv"
df_sample.to_csv(sample_csv, index=False)

# === Step 2: Load actual column names from saved CSV ===
column_names = pd.read_csv(sample_csv, nrows=0).columns.tolist()

# === Step 3: Run TANE ===
T = read_db(sample_csv)
tane = TANE(T)

start = time.time()
tane.run()
end = time.time()

# === Step 4: Print results safely ===
print(f"\n Execution Time: {end - start:.2f} seconds")
print(f" {len(tane.rules)} FDs found:\n")

for lhs, rhs in tane.rules:
    if rhs < len(column_names) and all(i < len(column_names) for i in lhs):
        lhs_names = [column_names[i] for i in lhs]
        rhs_name = column_names[rhs]
        print(f"{', '.join(lhs_names)} → {rhs_name}")



 Execution Time: 0.05 seconds
 42 FDs found:

Inspection ID → License #
Inspection ID → DBA Name
Inspection ID → Facility Type
Inspection ID → Risk
Inspection ID → City
Inspection ID → State
Inspection ID → Results
Risk → State
DBA Name → City
City → State
Facility Type → State
Results → State
DBA Name → State
License #, Results → City
License #, Risk → City
License #, State → City
License #, DBA Name → Risk
License #, Facility Type → City
License #, DBA Name → Facility Type
License #, Risk → Facility Type
License #, Risk, Results → DBA Name


In [38]:
import pandas as pd
import time
from itertools import combinations

def auto_fd_search(df, max_lhs=2):
    results = []
    start = time.time()
    columns = df.columns.tolist()

    for rhs in columns:
        lhs_candidates = [col for col in columns if col != rhs]
        for lhs in combinations(lhs_candidates, max_lhs):
            grouped = df.groupby(list(lhs))[rhs].nunique()
            if grouped.max() == 1:
                results.append((list(lhs), rhs))

    end = time.time()
    print(f"\n max_lhs = {max_lhs} |  Runtime: {end - start:.2f} seconds")
    print(f" {len(results)} functional dependencies found:\n")

    for lhs, rhs in results:
        print(f"{', '.join(lhs)} → {rhs}")
    return results


In [39]:
df = pd.read_csv("Food_Inspections_Sample.csv")

# 1 → 1 FDs
auto_fd_search(df, max_lhs=1)

# 2 → 1 FDs
auto_fd_search(df, max_lhs=2)

# 3 → 1 (optional but slower)
auto_fd_search(df, max_lhs=3)

# 4 → 1 (optional but slower)
auto_fd_search(df, max_lhs=4)



 max_lhs = 1 |  Runtime: 0.03 seconds
 15 functional dependencies found:

Inspection ID → License #
Inspection ID → DBA Name
Inspection ID → Facility Type
Inspection ID → Risk
Inspection ID → City
License # → City
DBA Name → City
Inspection ID → State
License # → State
DBA Name → State
Facility Type → State
Risk → State
City → State
Results → State
Inspection ID → Results

 max_lhs = 2 |  Runtime: 0.16 seconds
 70 functional dependencies found:

Inspection ID, DBA Name → License #
Inspection ID, Facility Type → License #
Inspection ID, Risk → License #
Inspection ID, City → License #
Inspection ID, State → License #
Inspection ID, Results → License #
Inspection ID, License # → DBA Name
Inspection ID, Facility Type → DBA Name
Inspection ID, Risk → DBA Name
Inspection ID, City → DBA Name
Inspection ID, State → DBA Name
Inspection ID, Results → DBA Name
Inspection ID, License # → Facility Type
Inspection ID, DBA Name → Facility Type
Inspection ID, Risk → Facility Type
Inspection ID, City

[(['Inspection ID', 'DBA Name', 'Facility Type', 'Risk'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Facility Type', 'City'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Facility Type', 'State'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Facility Type', 'Results'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Risk', 'City'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Risk', 'State'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Risk', 'Results'], 'License #'),
 (['Inspection ID', 'DBA Name', 'City', 'State'], 'License #'),
 (['Inspection ID', 'DBA Name', 'City', 'Results'], 'License #'),
 (['Inspection ID', 'DBA Name', 'State', 'Results'], 'License #'),
 (['Inspection ID', 'Facility Type', 'Risk', 'City'], 'License #'),
 (['Inspection ID', 'Facility Type', 'Risk', 'State'], 'License #'),
 (['Inspection ID', 'Facility Type', 'Risk', 'Results'], 'License #'),
 (['Inspection ID', 'Facility Type', 'City', 'State'], 'License #'),
 (['Inspection ID', 'Facility Type', 'Cit

In [40]:
import pandas as pd
import time
from itertools import combinations

# Step 1: Brute-force FD discovery
def auto_fd_search(df, max_lhs=2):
    results = []
    columns = df.columns.tolist()
    for rhs in columns:
        lhs_candidates = [col for col in columns if col != rhs]
        for lhs in combinations(lhs_candidates, max_lhs):
            grouped = df.groupby(list(lhs))[rhs].nunique()
            if grouped.max() == 1:
                results.append((list(lhs), rhs))
    return results

# Step 2: Filter for only minimal FDs
def filter_minimal_fds(fds):
    minimal = []
    for lhs, rhs in fds:
        is_minimal = True
        for other_lhs, other_rhs in fds:
            if rhs == other_rhs and set(other_lhs).issubset(set(lhs)) and lhs != other_lhs:
                is_minimal = False
                break
        if is_minimal:
            minimal.append((lhs, rhs))
    return minimal

# Step 3: Run across multiple LHS sizes and combine
def discover_minimal_fds(df, max_lhs=4):
    combined_results = []
    for i in range(1, max_lhs + 1):
        print(f" Searching FDs with max_lhs = {i} ...")
        result = auto_fd_search(df, max_lhs=i)
        combined_results.extend(result)
    minimal = filter_minimal_fds(combined_results)

    print(f"\n Final set of minimal FDs (after pruning redundant ones): {len(minimal)}\n")
    for lhs, rhs in minimal:
        print(f"{', '.join(lhs)} → {rhs}")
    return minimal

df = pd.read_csv("Food_Inspections_Sample.csv")
start_time = time.time()
minimal_fds = discover_minimal_fds(df, max_lhs=5)
total_time = time.time() - start_time

print(f"\n Total Runtime (all steps): {total_time:.2f} seconds")


 Searching FDs with max_lhs = 1 ...
 Searching FDs with max_lhs = 2 ...
 Searching FDs with max_lhs = 3 ...
 Searching FDs with max_lhs = 4 ...
 Searching FDs with max_lhs = 5 ...

 Final set of minimal FDs (after pruning redundant ones): 20

Inspection ID → License #
Inspection ID → DBA Name
Inspection ID → Facility Type
Inspection ID → Risk
Inspection ID → City
License # → City
DBA Name → City
Inspection ID → State
License # → State
DBA Name → State
Facility Type → State
Risk → State
City → State
Results → State
Inspection ID → Results
License #, DBA Name → Facility Type
License #, Risk → Facility Type
License #, DBA Name → Risk
Facility Type, Risk → City
License #, Risk, Results → DBA Name

 Total Runtime (all steps): 1.23 seconds


In [41]:
import pandas as pd

# Load your dataset
df = pd.read_csv("Food_Inspections_Cleaned.csv")

# Function to find unary INDs using a value index
def find_unary_inds_via_index(df):
    value_index = {}

    # Step 1: Build value index (value -> set of columns where it appears)
    for col in df.columns:
        for val in df[col].dropna().unique():
            if val not in value_index:
                value_index[val] = set()
            value_index[val].add(col)

    # Step 2: Check for unary INDs
    unary_inds = []
    for lhs in df.columns:
        lhs_values = df[lhs].dropna().unique()
        for rhs in df.columns:
            if lhs == rhs:
                continue
            # Check if every value in lhs also appears in rhs
            if all(rhs in value_index.get(val, set()) for val in lhs_values):
                unary_inds.append((lhs, rhs))
    return unary_inds

# Run the discovery and print results
inds = find_unary_inds_via_index(df)
print("Unary INDs found (LHS ⊆ RHS):")
for lhs, rhs in inds:
    print(f"{lhs} ⊆ {rhs}")


Unary INDs found (LHS ⊆ RHS):
